In [93]:
import pandas as pd
import numpy as np 
import os, sys
sys.path.append(os.getcwd())
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [94]:
cierre_agricola =pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/cierre_agricola.csv', encoding='latin1')
dfPP = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/PP_Chiapas.csv', encoding='latin1')
dfPT = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/dfPT.csv', encoding='latin1')
dfS = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/Suelos_Chiapas.csv', encoding='latin1')


In [95]:
cierre_agricola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22098 entries, 0 to 22097
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Anio                22098 non-null  int64  
 1   Idestado            22098 non-null  int64  
 2   Nomestado           22098 non-null  object 
 3   Idddr               22098 non-null  int64  
 4   Nomddr              22098 non-null  object 
 5   Idcader             22098 non-null  int64  
 6   Nomcader            22098 non-null  object 
 7   Idmunicipio         22098 non-null  int64  
 8   Nommunicipio        22098 non-null  object 
 9   Idciclo             22098 non-null  int64  
 10  Nomcicloproductivo  22098 non-null  object 
 11  Idmodalidad         22098 non-null  int64  
 12  Nommodalidad        22098 non-null  object 
 13  Idunidadmedida      22098 non-null  int64  
 14  Nomunidad           22098 non-null  object 
 15  Idcultivo           22098 non-null  int64  
 16  Nomc

In [96]:
df = cierre_agricola
# Filtrar el DataFrame para excluir las filas donde 'Nomcultivo' es 'Pastos y praderas'
df = df[df['Nomcultivo'] != 'Pastos y praderas']

# Verifica que se haya eliminado correctamente
print(df['Nomcultivo'].unique())

['MaÃ\xadz grano' 'SandÃ\xada' 'Sorgo grano' 'Cacahuate' 'Aguacate'
 'Ciruela' 'LimÃ³n' 'Mango' 'Naranja' 'Papaya' 'Frijol' 'CafÃ© cereza'
 'Mandarina' 'PlÃ¡tano' 'Cebolla' 'Chile verde' 'MelÃ³n' 'Guayaba' 'Mamey'
 'Zapote' 'Pimienta' 'Cacao' 'Nanche' 'Tomate rojo (jitomate)'
 'Tomate verde' 'MaraÃ±Ã³n' 'Tamarindo' 'CaÃ±a de azÃºcar' 'Durazno'
 'Manzana' 'Col (repollo)' 'Papa' 'PerÃ³n' 'Tejocote' 'Trigo grano' 'Lima'
 'Toronja (pomelo)' 'Zarzamora' 'Garbanzo grano' 'Pepino' 'Arroz palay'
 'Chile seco' 'Girasol' 'Hule hevea' 'Palma africana o de aceite'
 'Macadamia' 'Tabaco' 'AjonjolÃ\xad' 'Soya' 'Copra' 'PiÃ±a'
 'Calabaza semilla o chihua' 'Vainilla' 'Camote'
 'Sorgo forrajero en verde' 'Hortalizas' 'RambutÃ¡n'
 'CaÃ±a de azÃºcar piloncillo' 'Palma camedor' 'Pera' 'PiÃ±Ã³n'
 'Semilla de caÃ±a de azÃºcar' 'Litchi' 'Semilla de soya' 'Stevia'
 'Semilla de maÃ\xadz grano' 'MangostÃ¡n' 'Elote' 'Chayote' 'Granada'
 'Crisantemo (gruesa)' 'Lilium (gruesa)' 'Palma camedor (gruesa)']


In [97]:
df.rename(columns= {'Anio': 'Año' ,'Nomestado': 'Estado', 'Nommunicipio': 'Municipio', }, inplace=True)

/tmp/ipykernel_556/767272086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns= {'Anio': 'Año' ,'Nomestado': 'Estado', 'Nommunicipio': 'Municipio', }, inplace=True)


In [98]:
sin_pasto = df.groupby('Año').agg({
    'Volumenproduccion': 'sum',
    'Valorproduccion': 'sum', 
}).reset_index()

In [99]:
dfPT.rename(columns={'AÃ±o': 'Año', 'Anual': 'PromTemp_anual'}, inplace=True)

In [100]:
dfPP.rename(columns={'AÃ±o': 'Año', 'Anual': 'Prec_anual'}, inplace=True)

In [101]:
dfPP

,ID_AÃ±o,Año,ID_Estado,Estado,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Prec_anual
0,1.0,2003,7,Chiapas,23.9,13.5,35.6,26.5,145.4,314.6,232.9,309.8,371.5,253.4,155.4,53.1,1935.6
1,2.0,2004,7,Chiapas,45.7,42.3,31.1,42.8,193.8,250.2,249.1,195.7,310.8,235.2,69.1,48.0,1713.8
2,3.0,2005,7,Chiapas,8.5,19.7,33.8,47.4,158.4,413.4,350.8,338.9,362.7,315.0,56.8,85.3,2190.7
3,4.0,2006,7,Chiapas,78.4,47.3,32.2,49.2,229.7,447.0,356.7,285.1,339.4,323.5,94.4,71.3,2354.2
4,5.0,2007,7,Chiapas,104.0,31.6,33.4,58.5,148.0,256.6,279.3,459.1,368.3,503.2,41.4,19.2,2302.6
5,6.0,2008,7,Chiapas,69.9,50.6,33.3,90.8,203.2,431.3,390.0,315.8,486.0,224.2,25.8,34.0,2354.9
6,7.0,2009,7,Chiapas,47.0,18.5,29.0,27.0,168.1,369.0,214.5,333.7,300.0,245.9,142.0,58.4,1953.1
7,8.0,2010,7,Chiapas,45.1,17.8,14.8,70.4,186.4,507.5,395.1,756.7,559.5,58.4,89.9,28.7,2730.3
8,9.0,2011,7,Chiapas,46.0,37.1,84.3,74.2,149.5,300.6,458.5,433.7,422.3,273.7,58.8,42.9,2381.6
9,10.0,2012,7,Chiapas,53.3,38.6,58.2,57.8,181.1,264.0,211.8,418.1,321.3,194.4,29.0,57.7,1885.3


In [102]:
dfPP['Prom_anual']=dfPP['Prec_anual']/12

In [103]:
dfPP = dfPP[['Año','Prom_anual']]
dfPT = dfPT[['Año','PromTemp_anual']]

In [104]:
df_pred = pd.merge(sin_pasto, dfPP, on=['Año'], how='left')
df_pred = pd.merge(df_pred, dfPT, on=['Año'], how='left')

df_pred.head()

,Año,Volumenproduccion,Valorproduccion,Prom_anual,PromTemp_anual
0,2003,6417346.15,8.636436e+09,161.300000,24.0
1,2004,6195827.08,8.741246e+09,142.816667,24.0
2,2005,6210077.83,8.660470e+09,182.558333,24.7
3,2006,5971190.30,1.021940e+10,196.183333,25.4
4,2007,5963631.89,1.125255e+10,191.883333,25.0


In [105]:
X = df_pred[['Año', 'Valorproduccion','Prom_anual','PromTemp_anual']]
y = df_pred['Volumenproduccion']

In [106]:
#Predicciones para los proximos 5 años para el volumen de producciòn
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state =42)

X = df_pred[['Año', 'Valorproduccion','Prom_anual','PromTemp_anual']]
y = df_pred['Volumenproduccion']

#Creando y entrenando el modelo de regresion lineal
model = LinearRegression()
model.fit(X_train, y_train)

años_futuros = np.array([2024, 2025, 2026, 2027, 2028]).reshape(-1, 1)
X_futuro = pd.DataFrame({'Año': años_futuros.flatten(),
                         'Valorproduccion':np.mean(X_train['Valorproduccion']),
                          'Prom_anual': np.mean(X_train['Prom_anual']),
                          'PromTemp_anual': np.mean(X_train['PromTemp_anual'])})

predicciones = model.predict(X_futuro)

#Mostrar las predicciones
for Año, pred in zip(años_futuros.flatten(), predicciones):
    print(f'Predicciòn para el año {Año}: {pred:.2f} toneladas')

#Evaluacion del modelo 
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse: .2f}')




Predicciòn para el año 2024: 6316511.33 toneladas
Predicciòn para el año 2025: 6298472.66 toneladas
Predicciòn para el año 2026: 6280433.99 toneladas
Predicciòn para el año 2027: 6262395.33 toneladas
Predicciòn para el año 2028: 6244356.66 toneladas
MAE: 275386.16
RMSE:  395208.98
